# Seed2LP: Run for one solution per network to get time for first value

This notebook explain how to run Seed2lp to get the first solution, and must be run **AFTER** retrieving BiGG SBML file (see notebook [01_get_sbml_BiGG.ipynb](./01_get_sbml_BiGG.ipynb)) ant getting objective (see notebook [02_get_objectives.ipynb](./02_get_objectives.ipynb))

> Note:
>
> The Seed2lp (seed searching and flux) result files for one solution are available: [https://doi.org/10.57745/OS1JND](https://doi.org/10.57745/OS1JND)
>
> After downloadind and unzipping the package, go to analyses/results/one_solution

## **WARNING**
This notebook will run Seed2LP for all 107 GSMNs from BiGG the target seed searching mode, using methods: *Reasoning*, *Hybrid-Filter*, *Hybrid-GC*, *Hybrid-GC<sub>Div<sub>*.

No accumulation allowed NE seed inference, using subset minimal and minimize otptimization. Seed2LP is set to find 1 solution and will stop if exceed 10 min.

On the paper, the time limit is set to 45 min and number of solutions limit to 1.

To avoid a long time running within the notebook, the notebook will use a copy of e_coli_core in an sbml directory already created in notebook [03_run_seed2lp](03_run_seed2lp.ipynb).

## Requirements
Module *seed2lp* needed

> Advice:
> 
> Use a conda env called s2lp with python 3.10 for plafrim cluster scripts

In [ ]:
!pip install seed2lp

## **Slurm-based cluster**: Reproducing paper data
Slurm-based scripts for cluster are available with 45 min and 1000 for all networks:
- Launch if needed 
    - [01_job_retrieve_bigg_sbml.sh](../../scripts/plafrim_cluster/01_job_retrieve_bigg_sbml.sh): `sbatch 01_job_retrieve_bigg_sbml.sh`
    - [02_job_get_objective.sh](../../scripts/plafrim_cluster/02_job_get_objective.sh): `sbatch 02_job_get_objective.sh`
    - or copy your local files into you cluster
- Change **_source_** variable by the path of your conda environement with seed2lp installed in files: [03_execute_workflow_search.sh](../../scripts/plafrim_cluster/03_execute_workflow_search.sh) and [03_job_run_s2lp.sh](../../scripts/plafrim_cluster/01_job_run_s2lp.sh) 
- launch [03_execute_workflow_search.sh](../../scripts/plafrim_cluster/03_execute_workflow_search.sh): `sbatch 03_execute_workflow_search.sh`

## **LAUNCH**

### Variable to change (if wanted)

In [1]:
analyse_dir = "../../analyses"
data_dir  = f"{analyse_dir}/data/"
result_dir=f"{analyse_dir}/results"
temp_dir = "../../tmp/"

time_limit = 10 # time limit
number_solution = 30 # number solutions

### Execute

In [2]:
from os import path, listdir, makedirs
from shutil import copyfile

In [ ]:
sbml_dir = f"{data_dir}/bigg/sbml"
e_coli_dir = f"{data_dir}/bigg/sbml_e_coli_core"
result_dir =  f"{result_dir}/one_solution"
objecive_dir = f"{data_dir}/objective"

In [3]:
if not path.isdir(e_coli_dir):
    makedirs(e_coli_dir)
    copyfile(path.join(sbml_dir, "e_coli_core.xml"), path.join(e_coli_dir, "e_coli_core.xml"))

This function will execute seed2lp for iCN718:
- Target
- subset minimal
- *Reasoning*, *Hybrid-Filter*, *Hybrid-GC* and *Hybrid-GC<sub>Div<sub>*
- no accumulation
- maximisation (of flux in Objective reaction)
- Limitations: 30 solutions and 10 min

Also, it will check the flux for each solution and write it into files.

In [7]:
def run_s2lp(in_dir:str):
    for filename in listdir(in_dir):
        species = f'{path.splitext(path.basename(filename))[0]}'
        sbml_path = path.join(in_dir,f"{species}.xml")
        objective_path = path.join(objecive_dir,f"{species}_target.txt")
        result_path = path.join(result_dir,species)

        command = f"target {sbml_path} {result_path} --temp {temp_dir} -tl {time_limit} -nbs {number_solution} -cf -max -tf {objective_path}"

        command_reasoning=command+' -so reasoning'
        command_filter=command+' -so filter'
        command_guess_check=command+' -so guess_check'
        command_guess_check_div=command+' -so guess_check_div'

        !seed2lp {command_reasoning}
        !seed2lp {command_filter}
        !seed2lp {command_guess_check}
        !seed2lp {command_guess_check_div}

The execution might take more than 3min due to finding minimal solutions and due to *Hybrid-lpx* mode (requires lot of time to calculate fluxes)

In [8]:
run_s2lp(e_coli_dir)

           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
                                    |_|         
      
Network name: e_coli_core

____________________________________________

                  TARGETS                   
          FOR TARGET MODE AND FBA           
____________________________________________

Targets set:
    Reactant of objective reaction
    from target file


____________________________________________

                  OBJECTVE                  
                 FOR HYBRID                 
____________________________________________

Objective set:
    Objective reaction from target file


Objective : R_BIOMASS_Ecoli_core_w_GAM



____________________________________________

                  NETWORK                   
____________________________________________

Import reaction:

### **List of output files**

In the result directory (initially "../../results/one_solution/e_coli_core") you will find 8 files.

Seed2lp results files:
- e_coli_core_rm_rxn_tgt_taf_reas_max_no_accu_results.json -> Reasoning
- e_coli_core_rm_rxn_tgt_taf_fil_max_no_accu_results.json -> Filter
- e_coli_core_rm_rxn_tgt_taf_gc_max_no_accu_results.json -> Guess-Check
- e_coli_core_rm_rxn_tgt_taf_gcd_max_no_accu_results.json -> Guess-Check and Diversity

Fluxes files:
- e_coli_core_rm_rxn_tgt_taf_reas_max_no_accu_fluxes.tsv -> Reasoning
- e_coli_core_rm_rxn_tgt_taf_fil_max_no_accu_fluxes.tsv -> Filter
- e_coli_core_rm_rxn_tgt_taf_gc_max_no_accu_fluxes.tsv -> Guess-Check
- e_coli_core_rm_rxn_tgt_taf_gcd_max_no_accu_fluxes.tsv -> Guess-Check and Diversity

> Note:
>
> You will find log files in [result_directory]/one_solution/e_coli_core/logs
>
> Example: ../../results/one_solution/e_coli_core/logs